<a href="https://colab.research.google.com/github/nlakshmisoundarya/MY-PRACTICES/blob/main/Employee_Attendance_and_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, date


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/employee_data.csv")
column_names = df.columns.tolist()
print("Column names:", column_names)
print(df.head())

print("\nDataset information:")
print(df.info())

print("\nSummary statistics:")
print(df.describe())

print("\nMissing values:")
print(df.isnull().sum())

Column names: ['employee_id', 'date', 'status', 'in_time', 'in_time_delay', 'out_time', 'out_time_delay', 'duration']
  employee_id        date   status   in_time  in_time_delay  out_time  \
0    6JD0R6MD  01/01/2023  holiday       NaN            NaN       NaN   
1    6JD0R6MD  01/02/2023      NaN  10:02 AM            NaN  05:27 PM   
2    6JD0R6MD  01/03/2023      NaN  10:29 AM            NaN  05:43 PM   
3    6JD0R6MD  01/04/2023      NaN  10:51 AM            NaN  06:44 PM   
4    6JD0R6MD  01/05/2023      NaN  10:35 AM            NaN  06:13 PM   

   out_time_delay  duration  
0             NaN       NaN  
1             NaN       NaN  
2             NaN       NaN  
3             NaN       NaN  
4             NaN       NaN  

Dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3360000 entries, 0 to 3359999
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   employee_id     object 
 1   date            object 
 2   status  

Marking Status coloumn


In [ ]:
df['date'] = pd.to_datetime(df['date'])
def is_weekend(date):
    return date.weekday() in [5, 6]
df['status'] = df.apply(lambda row: 'Holiday' if is_weekend(row['date']) else row['status'], axis=1)
print(df)

        employee_id       date   status   in_time  in_time_delay  out_time  \
0          6JD0R6MD 2023-01-01  Holiday       NaN            NaN       NaN   
1          6JD0R6MD 2023-01-02     Late  10:02 AM            2.0  05:27 PM   
2          6JD0R6MD 2023-01-03     Late  10:29 AM           29.0  05:43 PM   
3          6JD0R6MD 2023-01-04     Late  10:51 AM           51.0  06:44 PM   
4          6JD0R6MD 2023-01-05     Late  10:35 AM           35.0  06:13 PM   
...             ...        ...      ...       ...            ...       ...   
3359995    1EPT0JR4 2023-12-24  Holiday       NaN            NaN       NaN   
3359996    1EPT0JR4 2023-12-25     Late  11:29 AM           89.0  06:13 PM   
3359997    1EPT0JR4 2023-12-26     Late  10:39 AM           39.0  05:03 PM   
3359998    1EPT0JR4 2023-12-27  Present  09:26 AM            0.0  06:47 PM   
3359999    1EPT0JR4 2023-12-28     Late  10:05 AM            5.0  06:01 PM   

         out_time_delay  duration  month  
0                   

In [ ]:
nan_count = df['out_time'].isna().sum()
print(nan_count)

960000


Updating status coloumn


In [ ]:
def update_status(row):
    if pd.isna(row['status']):
        if pd.notna(row['in_time']):
            in_time = pd.to_datetime(row['in_time'], format='%I:%M %p').time()
            if in_time < pd.to_datetime('09:00 AM', format='%I:%M %p').time():
                return 'Present'
            else:
              return 'Late'
        elif pd.isna(row['in_time']):
            if pd.isna(row['out_time']):
                return 'Absent'
            else:
                return 'Late'
    return row['status']
df['status'] = df.apply(update_status, axis=1)
print(df)

        employee_id       date   status   in_time  in_time_delay  out_time  \
0          6JD0R6MD 2023-01-01  Holiday       NaN            NaN       NaN   
1          6JD0R6MD 2023-01-02     Late  10:02 AM            2.0  05:27 PM   
2          6JD0R6MD 2023-01-03     Late  10:29 AM           29.0  05:43 PM   
3          6JD0R6MD 2023-01-04     Late  10:51 AM           51.0  06:44 PM   
4          6JD0R6MD 2023-01-05     Late  10:35 AM           35.0  06:13 PM   
...             ...        ...      ...       ...            ...       ...   
3359995    1EPT0JR4 2023-12-24  Holiday       NaN            NaN       NaN   
3359996    1EPT0JR4 2023-12-25     Late  11:29 AM           89.0  06:13 PM   
3359997    1EPT0JR4 2023-12-26     Late  10:39 AM           39.0  05:03 PM   
3359998    1EPT0JR4 2023-12-27  Present  09:26 AM            0.0  06:47 PM   
3359999    1EPT0JR4 2023-12-28     Late  10:05 AM            5.0  06:01 PM   

         out_time_delay  duration  month  
0                   

Calculating In Time Delay


In [ ]:
from datetime import datetime, time
def update_in_time_delay(row):
    if row['status'] in ['Absent', 'Holiday']:
        return row['in_time_delay']
    elif row['status'] == 'Present':
        return 0
    elif row['status'] == 'Late' and pd.notna(row['in_time']):
        in_time = datetime.strptime(row['in_time'], '%I:%M %p').time()
        scheduled_time = time(9, 0)
        delay_minutes = (datetime.combine(datetime.today(), in_time) - datetime.combine(datetime.today(), scheduled_time)).seconds // 60
        return delay_minutes
    else:
        return row['in_time_delay']
df['in_time_delay'] = df.apply(update_in_time_delay, axis=1)
print(df)

        employee_id       date   status   in_time  in_time_delay  out_time  \
0          6JD0R6MD 2023-01-01  Holiday       NaN            NaN       NaN   
1          6JD0R6MD 2023-01-02     Late  10:02 AM           62.0  05:27 PM   
2          6JD0R6MD 2023-01-03     Late  10:29 AM           89.0  05:43 PM   
3          6JD0R6MD 2023-01-04     Late  10:51 AM          111.0  06:44 PM   
4          6JD0R6MD 2023-01-05     Late  10:35 AM           95.0  06:13 PM   
...             ...        ...      ...       ...            ...       ...   
3359995    1EPT0JR4 2023-12-24  Holiday       NaN            NaN       NaN   
3359996    1EPT0JR4 2023-12-25     Late  11:29 AM          149.0  06:13 PM   
3359997    1EPT0JR4 2023-12-26     Late  10:39 AM           99.0  05:03 PM   
3359998    1EPT0JR4 2023-12-27  Present  09:26 AM            0.0  06:47 PM   
3359999    1EPT0JR4 2023-12-28     Late  10:05 AM           65.0  06:01 PM   

         out_time_delay  duration  month  
0                   

Calculating Out Time Delay

In [ ]:
from datetime import datetime, time
def update_out_time_delay(row):
    if row['status'] in ['Absent', 'Holiday']:
        return row['out_time_delay']
    elif row['status'] == 'Present':
        return 0
    elif row['status'] == 'Late' and pd.notna(row['out_time']):
        out_time = datetime.strptime(row['out_time'], '%I:%M %p').time()
        scheduled_time = time(18, 00)
        if out_time < scheduled_time:
            delay_minutes = (scheduled_time.hour * 60 + scheduled_time.minute) - (out_time.hour * 60 + out_time.minute)
            return delay_minutes
        else:
            return 0
    else:
        return row['out_time_delay']

df['out_time_delay'] = df.apply(update_out_time_delay, axis=1)
print(df)

        employee_id       date   status   in_time  in_time_delay  out_time  \
0          6JD0R6MD 2023-01-01  Holiday       NaN            NaN       NaN   
1          6JD0R6MD 2023-01-02     Late  10:02 AM           62.0  05:27 PM   
2          6JD0R6MD 2023-01-03     Late  10:29 AM           89.0  05:43 PM   
3          6JD0R6MD 2023-01-04     Late  10:51 AM          111.0  06:44 PM   
4          6JD0R6MD 2023-01-05     Late  10:35 AM           95.0  06:13 PM   
...             ...        ...      ...       ...            ...       ...   
3359995    1EPT0JR4 2023-12-24  Holiday       NaN            NaN       NaN   
3359996    1EPT0JR4 2023-12-25     Late  11:29 AM          149.0  06:13 PM   
3359997    1EPT0JR4 2023-12-26     Late  10:39 AM           99.0  05:03 PM   
3359998    1EPT0JR4 2023-12-27  Present  09:26 AM            0.0  06:47 PM   
3359999    1EPT0JR4 2023-12-28     Late  10:05 AM           65.0  06:01 PM   

         out_time_delay  duration  month  
0                   

Calculating Duration Coloumn


In [ ]:
def update_duration(row):
    if pd.notna(row['in_time_delay']) and pd.notna(row['out_time_delay']):
        if row['in_time_delay'] == 0.0 and row['out_time_delay'] == 0.0:
            return '9.00'
        else:
            total_minutes = (9 * 60) - (row['in_time_delay'] + row['out_time_delay'])
            hours = total_minutes // 60
            minutes = total_minutes % 60
            return f'{int(hours)}.{int(minutes):02}'
    else:
        return None
df['duration'] = df.apply(update_duration, axis=1)
print(df)

        employee_id       date   status   in_time  in_time_delay  out_time  \
0          6JD0R6MD 2023-01-01  Holiday       NaN            NaN       NaN   
1          6JD0R6MD 2023-01-02     Late  10:02 AM           62.0  05:27 PM   
2          6JD0R6MD 2023-01-03     Late  10:29 AM           89.0  05:43 PM   
3          6JD0R6MD 2023-01-04     Late  10:51 AM          111.0  06:44 PM   
4          6JD0R6MD 2023-01-05     Late  10:35 AM           95.0  06:13 PM   
...             ...        ...      ...       ...            ...       ...   
3359995    1EPT0JR4 2023-12-24  Holiday       NaN            NaN       NaN   
3359996    1EPT0JR4 2023-12-25     Late  11:29 AM          149.0  06:13 PM   
3359997    1EPT0JR4 2023-12-26     Late  10:39 AM           99.0  05:03 PM   
3359998    1EPT0JR4 2023-12-27  Present  09:26 AM            0.0  06:47 PM   
3359999    1EPT0JR4 2023-12-28     Late  10:05 AM           65.0  06:01 PM   

         out_time_delay duration  month  
0                   N

Month Wise Analysis

In [ ]:
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
df['month'] = pd.to_datetime(df['date']).dt.month
monthly_analysis = df.groupby(['employee_id', 'month'])['duration'].mean().reset_index()
print(monthly_analysis)

       employee_id  month  duration
0         0002UVE4      1    7.3860
1         0002UVE4      2    7.4655
2         0002UVE4      3    7.2925
3         0002UVE4      4    7.3485
4         0002UVE4      5    7.4380
...            ...    ...       ...
119995    ZZ71FLPO      8    7.1965
119996    ZZ71FLPO      9    7.1215
119997    ZZ71FLPO     10    7.0650
119998    ZZ71FLPO     11    7.2560
119999    ZZ71FLPO     12    7.0690

[120000 rows x 3 columns]


In [ ]:
!pip install dash dash-core-components dash-html-components plotly

import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px


app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Employee Performance Dashboard"),

    dcc.Dropdown(
        id='employee-dropdown',
        options=[{'label': emp, 'value': emp} for emp in df['employee_id'].unique()],
        value=df['employee_id'].iloc[0],
        style={'width': '50%'}
    ),

    dcc.Graph(id='monthly-attendance'),

    dcc.Graph(id='late-arrivals'),

    dcc.Graph(id='performance-summary'),
])


@app.callback(
    [Output('monthly-attendance', 'figure'),
     Output('late-arrivals', 'figure'),
     Output('performance-summary', 'figure')],
    [Input('employee-dropdown', 'value')]
)
def update_graphs(selected_employee):
    employee_df = df[df['employee_id'] == selected_employee]

    monthly_attendance_fig = px.line(
        employee_df, x='date', y='duration', title='Monthly Attendance Trend'
    )
    late_arrivals_fig = px.bar(
        employee_df[employee_df['status'] == 'Late'],
        x='date', y='in_time_delay', title='Late Arrivals'
    )

    performance_summary_fig = px.pie(
        employee_df, names='status', title='Performance Summary'
    )

    return monthly_attendance_fig, late_arrivals_fig, performance_summary_fig

app.run_server(debug=True, use_reloader=False)


<IPython.core.display.Javascript object>